In [1]:
%load_ext autoreload
%autoreload 2

# Homework: Competitive Grammar Writing

In [2]:
from pcfg_parse_gen import Pcfg, PcfgGenerator, CkyParse
import os
import pandas as pd
from nltk import *
import nltk
from collections import Counter
import nltk.tokenize.punkt
import pickle
import csv
import json
import codecs
import string
import re

def print_tree(tree_string):
    tree_string = tree_string.strip()
    tree = nltk.Tree.fromstring(tree_string)
    tree.pretty_print()

def draw_tree(tree_string):
    tree_string = tree_string.strip()
    tree = nltk.Tree.fromstring(tree_string)
    tree.draw()

##### External Packages: 

In [4]:
from stanfordcorenlp import StanfordCoreNLP
# https://stanfordnlp.github.io/CoreNLP/
sf_nlp_core = StanfordCoreNLP(r'stanford-corenlp-full-2018-02-27')

from nltk.tag import StanfordPOSTagger, StanfordNERTagger
# https://nlp.stanford.edu/software/tagger.shtml
pos_jar = r'stanford-postagger-full-2018-02-27/stanford-postagger.jar'
pos_model = r'stanford-postagger-full-2018-02-27/models/english-left3words-distsim.tagger'
sf_pos_tagger = StanfordPOSTagger(pos_model, pos_jar, encoding='utf8')

import benepar
benepar.download('benepar_en')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package benepar_en to
[nltk_data]     C:\Users\jacky\AppData\Roaming\nltk_data...
[nltk_data]   Package benepar_en is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jacky\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jacky\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Contents  <a id='part-0'>
-------------------
- [Part 1: Data Collection - Generate more sentances](#part-1)
- [Part 2: Data Preprocessing - Tagging](#part-2)
- [Part 3: Grammer Generation - S1](#part-3)
- [Part 4: Grammer Generation - S2](#part-4)
- [Part 5: Parsing sentances](#part-5)
- [Part 6: Generating sentences](#part-6)

# Part 1. Data Collection - Generate more sentances <a id='part-1'></a>

### Find and process more sentences from the same distribution as the devset

#### sentences source: Monty Python and The Holy Grail
#### http://www.castlebar.ie/board/msgjan2001/22833.htm

**Steps:**
>**1. Manually copy new sentences to a local file.**<br>
>**2. Correct wrong person names and some formating issues both programmingly and manually.**<br>
>**3. Check if all the words is in the allowed words file.**<br>

Correct wrong person names and some formating issues. <br>
Manual replacements:
>Tiogar -> Zoot <br>
>Idiotic trivia -> Concorde <br>
>melmacs -> keepers <br>
>melmac -> keeper <br>
>Melmac -> Old man <br>
>Times. -> Tim the Enchanter <br>
>Fortycoats -> Maynard <br>
>Jesus el christos -> Father <br>
>Baloo -> Herbert <br>
>Chaz -> Dingo <br>
>illegitimes .Dougalate faced -> illegitimate faced <br>

In [6]:
pruncList = ['.', '?', '!', ',', '-', "'"]
file3 = open('quotes_new.txt', 'r')
file4 = open('quotes_new_preprocessed.txt', 'w')
for i in file3:
    if not (i.startswith("Scene") or i.startswith("[")) :
        pos = i.find(':')
        txt = i.strip()
        wrt = ''
        if (pos != -1) and (len(txt)!=0) and (txt[-1]!=':'):
            wrt = i[pos+1:].strip()
        else:
            wrt = i.strip()
        if (len(txt)!=0) and (wrt[-1] not in pruncList):
            wrt = wrt+' '
            
        wrt = wrt.replace('...', ' $$$\n').replace('....', ' $$$$\n').replace('.', ' .\n')\
        .replace('!', ' !\n').replace('?', ' ?\n').replace('\n ', '\n')\
        .replace(',', ' ,').replace('$', '.').replace(' -- ', '--').replace('--', ' -- ')\
        .replace('Bulletin board posters', 'Britons').replace('Scrawny', 'Arthur')\
        .replace('Voodoo Child', 'Patsy').replace('Pete Jordan', 'Dennis')\
        .replace('Kharn', 'Bedemir').replace('Fr ', 'times').replace('Moderator', 'God')\
        .replace('jesus el christos', 'father').replace('Luca Brasi', 'Launcelot')\
        .replace('Buzz', 'Galahad').replace('Spartacus', 'Robin').replace('Dougales', '')\
        .replace('EN', 'Black Knight').replace('Dougale', '')\
        .replace("'d", " 'd").replace("'em", " 'em").replace("'ll", " 'll")\
        .replace("'ow", " 'ow").replace("'re", " 're").replace("'s", " 's").replace("'ve", " 've")\
        .replace("n't", " n't").replace("y'", "y' ").replace("d'", "d' ").replace("`", "` ")\
        .replace("we 're", "we're").replace("We 're", "We're").replace("You 're", "You're")\
        .replace("you 're", "you're").replace("We 've", "We've").replace("we 've", "we've")\
        .replace("I 've", "I've").replace("ca n't", "can't")
        
        if('--' not in wrt):
            wrt = wrt.replace('-', ' - ')
        file4.write(wrt)

file3.close()
file4.close()

Check if all the words are in the allowed words file.

In [ ]:
allowed_words = []
file6 = open('allowed_words.txt', 'r')
for i in file6:
    allowed_words.append(i.strip())
file6.close()

count = 0
checked = True
file5 = open('quotes_new_preprocessed.txt', 'r')
for i in file5:
    sentence = i.strip().split(' ')
    count+=1
    for word in sentence:
        word2 = word.strip()
        if (word2 != '') and (word2 not in allowed_words):
            print(count)
            print(word2)
            checked = False
file5.close()
if checked:
    print("All words are in the allowed words file. ")

###  [Back to contents](#part-0)

# Part 2. Data Preprocessing - Tagging  <a id='part-2'></a>

### Phase 1. Preparation

In [8]:
def save(path, data):
    if '.p' in path or '.pickle' in path:
        save_pickle(path, data)
    else:
        raise NotImplementedError()
        
def load(path):
    if '.p' in path or '.pickle' in path:
        return load_pickle(path)
    else:
        raise NotImplementedError()
    
def save_pickle(path, data):
    with open(path, 'wb') as f:
        pickle.dump(data, f)

def load_pickle(path):
    result = None
    with open(path, 'rb') as f:
        result = pickle.load(f)
    return result

### Phase 2. Load Allowed Words and change existing tag to Complex POS format

In [9]:
words = []
with open('allowed_words.txt', 'r') as f:
    for lines in f:
        words.append(lines.replace('\n', ''))

awords = pd.DataFrame({'words':words})

In [10]:
awords.sample(frac=1).head()

,words
408,When
218,In
532,bangin
1511,shh
261,My


##### Sperater set to be 4 consecative space, change the file if this is not the case

In [11]:
t = pd.read_csv('Vocab.gr', sep='    ', comment='#', header=None, engine='python', names=['p','tag','words'])
j = awords.merge(t, on='words', how='left')
j = j.drop_duplicates()

In [12]:
j.sample(frac=1).head()

,words,p,tag
1026,hear,NaN,NaN
1875,weary,1.0,Misc
568,behind,NaN,NaN
1737,third,NaN,NaN
1675,supports,NaN,NaN


#### Map our old tag to new POS tag

In [13]:
map_to_pos = {
    'VerbT': 'c', 
    'Number': 'CD', 
    'Proper': 'NNP',  # or NNPS
    'Det': 'DT',
    'Prep': 'IN', 
    'Noun': 'NN' # or NNS
}
can_map = j['tag'].isin(map_to_pos.keys())
j.loc[can_map, 'tag'] = j.loc[can_map, 'tag'].apply(lambda x : map_to_pos[x])

In [14]:
j['first'] = j['words'].str[0]

is_shorten = j['words'].str.startswith("'") & (j['words'] != "'")
is_number = j['first'].str.isdigit()
is_punc = j['first'].isin(set(string.punctuation)) & ~((j['first'] == "'") & (j['words'].str.len() > 1))

# Default the shorten form to "Verb, 3rd person singular present"
# It may be wrong, but we can update it using parsed data
j.loc[is_shorten, 'tag'] = 'VBZ'
j.loc[is_number,'tag'] = 'CD'
# Punc can be { . , : '' ``}
# . end of line
# , pause
# : conjunction or end of line
# '' just '
# `` just `
# *** We will fix this later ***
j.loc[is_punc, 'tag'] = '.' 

In [15]:
j[(is_shorten)| (is_number) | (is_punc)].sample(frac=1).head()

,words,p,tag,first
13,'re,1.0,VBZ,'
441,`,NaN,.,`
7,'m,1.0,VBZ,'
19,'ve,NaN,VBZ,'
29,...,1.0,.,.


### Phase 3. Tagging with POS and Name Entity for single word
---------------------
in case we dont have enough sample

In [16]:
# Find the tags and entity information from StanfordNLP
j.loc[(~is_shorten) & (~is_number) & (~is_punc), 'sftag'] = \
    j[(~is_shorten) & (~is_number) & (~is_punc)]['words'].apply(lambda x: sf_nlp_core.pos_tag(x)[0][1])
j.loc[(~is_shorten) & (~is_number) & (~is_punc), 'entity'] = \
    j[(~is_shorten) & (~is_number) & (~is_punc)]['words'].apply(lambda x: sf_nlp_core.ner(x)[0][1])

In [17]:
j.sample(frac=0.8).head()

,words,p,tag,first,sftag,entity
111,Bread,NaN,NaN,B,NN,O
1305,nibble,NaN,NaN,n,VB,O
833,eh,NaN,NaN,e,UH,O
1660,successful,NaN,NaN,s,JJ,O
63,Almighty,NaN,NaN,A,NNP,O


### Phase 4. Word Segmentation & Tagging from samples

In [18]:
def load_sent(paths=[]):
    tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()
    if isinstance(paths, str):
        paths = [paths]
    if len(paths) > 0:
        text = ""
        for path in paths:
            text += codecs.open(path, "r" , "utf8").read()
    return tokenizer.tokenize(text.strip())

examples = load_sent(['devset.txt', 'quotes_new_preprocessed.txt','example_sentences.txt'])

In [19]:
# Remove duplication?
# examples = set(examples)

In [20]:
examples[:10]

['Whoa there !',
 'Halt !',
 'Who goes there ?',
 'It is I , Arthur , son of Uther Pendragon , from the castle of Camelot .',
 'King of the Britons , defeator of the Saxons , sovereign of all England !',
 'Pull the other one !',
 'I am .',
 'And this my trusty servant Patsy .',
 'We have ridden the length and breadth of the land in search of knights who will join me in my court of Camelot .',
 'I must speak with your lord and master .']

In [21]:
sf_tags = []
examples_parsed = []
try:
    sf_tags, examples_parsed = load('sf_tags_mp_allow.p')
except:
    for example in examples:
        word = word_tokenize(example)
        p = sf_pos_tagger.tag(word)
        sf_tags += p
        examples_parsed.append(p)
    # Cache...
    save('sf_tags_mp_allow.p', (sf_tags, examples_parsed))

In [22]:
df_sf_tags = pd.DataFrame({'words': [i[0] for i in sf_tags], 'sftag':[i[1] for i in sf_tags], 'dummy': 1})
df_sf_tags['counts'] = df_sf_tags.groupby(['words', 'sftag']).transform('count')
# we count every word + pos
df_sf_tags = df_sf_tags.drop_duplicates().reset_index(drop=True)
del df_sf_tags['dummy']

In [23]:
j = j.merge(df_sf_tags.rename(columns={'sftag':'parsed_sftag'}), on=['words'], how='left')

In [24]:
j.sample(frac=0.8).head()

,words,p,tag,first,sftag,entity,parsed_sftag,counts
1627,shrubberies,NaN,NaN,s,NNS,O,NN,1.0
1214,like,NaN,NaN,l,IN,O,VBP,2.0
18,--,1.0,.,-,NaN,NaN,:,67.0
1923,want,NaN,NaN,w,VB,O,VB,5.0
1956,wha,NaN,NaN,w,NN,O,NaN,NaN


#### If tag & sftag & parsed_sftag matched, we think its a good tag

In [25]:
is_same_tag = (j['sftag']==j['parsed_sftag']) & ((j['tag'].isnull()) | (j['tag'] == 'Misc'))
j.loc[is_same_tag, 'new_tag'] = j['parsed_sftag']

In [26]:
j[is_same_tag].sample(frac=0.8).head()

,words,p,tag,first,sftag,entity,parsed_sftag,counts,new_tag
1447,pissing,NaN,NaN,p,VBG,O,VBG,1.0,VBG
52,Ah,NaN,NaN,A,NN,O,NN,2.0,NN
1034,handed,NaN,NaN,h,NN,O,NN,1.0,NN
48,Ages,NaN,NaN,A,NNPS,O,NNPS,1.0,NNPS
1840,tops,NaN,NaN,t,NNS,O,NNS,1.0,NNS


#### If it is a frequent one, we think its pretty good tag

In [27]:
COUNT_THRESHOLD = 5
is_frequent = j['counts'] >= COUNT_THRESHOLD
j.loc[is_frequent, 'new_tag'] = j['parsed_sftag']

In [28]:
is_matched = (j['tag'] == j['sftag']) & (j['tag'] == j['parsed_sftag'])
j.loc[is_matched, 'new_tag'] = j['parsed_sftag']

In [29]:
j[is_matched].sample(frac=0.8).head()

,words,p,tag,first,sftag,entity,parsed_sftag,counts,new_tag
1794,the,1.0,DT,t,DT,O,DT,192.0,DT
314,Patsy,1.0,NNP,P,NNP,PERSON,NNP,5.0,NNP
453,Zoot,1.0,NNP,Z,NNP,O,NNP,8.0,NNP
969,from,1.0,IN,f,IN,O,IN,10.0,IN
1603,servant,1.0,NN,s,NN,TITLE,NN,1.0,NN


#### Add both sftag & parsed_sftag to new_tag

In [30]:
not_same = (j['new_tag'].isnull()) & (~j['sftag'].isnull()) & (~j['parsed_sftag'].isnull()) & (j['sftag'] != j['parsed_sftag'])
temp = j[not_same].copy()
temp['new_tag'] = temp['sftag']
j.loc[not_same, 'new_tag'] = j.loc[not_same, 'parsed_sftag']
j = j.append([temp], ignore_index=True)

#### Add the remining tag to new_tag

In [31]:
j.loc[j['new_tag'].isnull(), 'new_tag'] = j.loc[j['new_tag'].isnull(), 'sftag']

In [32]:
j.sample(frac=0.8).head()

,words,p,tag,first,sftag,entity,parsed_sftag,counts,new_tag
1044,has,1.0,c,h,VBZ,O,VBZ,8.0,VBZ
1011,grow,1.0,Misc,g,VB,O,VB,1.0,VB
439,Winter,NaN,NaN,W,NNP,DATE,NNP,3.0,NNP
2154,got,NaN,NaN,g,VBD,O,VBN,3.0,VBD
1081,himself,NaN,NaN,h,PRP,O,PRP,1.0,PRP


### Phase 5. Special Case Handling

#### Special Case - Puncuations

In [33]:
is_punc = (j['first'].isin(set(string.punctuation))) & ~((j['first'] == "'") & (j['words'].str.len() > 1))

def set_punc(s):
    import re
    if s in ['(']:
        return 'BGNBK'
    if s in [')']:
        return 'ENDBK'
    if s in ['"', "'"]:
        return 'QUOTE'
    if re.match('`+$', s):
        return 'QUOTE'
    if s in [':',';']:
        return 'BREAK'
    if s in ['.','!','?']:
        return 'END'
    if s in [',']:
        return 'PAUSE'
    if re.match('(\.\.+)$|(-+)$', s):
        return 'BREAK'
    raise Exception("{} not in cases".format(s))
j.loc[is_punc, 'new_tag'] = j.loc[is_punc, 'words'].apply(lambda x: set_punc(x))

In [34]:
j[is_punc].sample(frac=1).head()

,words,p,tag,first,sftag,entity,parsed_sftag,counts,new_tag
20,.,1.0,.,.,NaN,NaN,.,349.0,END
32,?,1.0,.,?,NaN,NaN,.,104.0,END
18,--,1.0,.,-,NaN,NaN,:,67.0,BREAK
21,..,NaN,.,.,NaN,NaN,NaN,NaN,BREAK
2,',NaN,.,',NaN,NaN,'',41.0,QUOTE


#### Special Case - Numbers

In [35]:
is_number = (j['new_tag'].isnull()) & ((j['tag'] == 'CD')| (j['tag']=='Number') | (j['entity']=='NUMBER'))
j.loc[is_number, 'new_tag'] = 'CD'

In [36]:
j[((j['tag'] == 'CD')| (j['tag']=='Number') | (j['entity']=='NUMBER'))].sample(frac=1).head()

,words,p,tag,first,sftag,entity,parsed_sftag,counts,new_tag
1865,twenty,NaN,NaN,t,CD,NUMBER,CD,2.0,CD
25,24,NaN,CD,2,NaN,NaN,CD,2.0,CD
1816,three,NaN,NaN,t,CD,NUMBER,CD,6.0,CD
1607,seven,1.0,Misc,s,CD,NUMBER,NaN,NaN,CD
935,five,1.0,Misc,f,CD,NUMBER,CD,6.0,CD


#### Special Case - Shorten words
-------------
Example:   
- 'ow -> how   
- 'em -> them   

In [37]:
is_shorten = (j['new_tag'].isnull()) & (j['first'] == "'") & (j['words'].str.len() > 1)
def set_shorten(s):
    if s == "'ow":
        return 'RB'
    if s == "'em":
        return 'PRP'
    return 'PRP'
j.loc[is_shorten, 'new_tag'] = j.loc[is_shorten, 'words'].apply(lambda x: set_shorten(x) )

In [38]:
j[(j['first'] == "'") & (j['words'].str.len() > 1)].sample(frac=1).head()

,words,p,tag,first,sftag,entity,parsed_sftag,counts,new_tag
4,'d,1.0,VBZ,',NaN,NaN,MD,6.0,MD
10,'s,1.0,VBZ,',NaN,NaN,VBZ,55.0,VBZ
9,'re,1.0,VBZ,',NaN,NaN,VBP,22.0,VBP
12,'s,1.0,VBZ,',NaN,NaN,POS,10.0,POS
6,'ll,1.0,VBZ,',NaN,NaN,MD,10.0,MD


#### Special Case - Shorten words
---------
Example:
- Aaaaagh, Aaauggh, Aah
- ug, uh, um
- Ooh, Oof

In [39]:
# Minior Cases - Specail UTs
UT_list = [
    '[A|a][a|u|g]*h+$', # Aaaaagh, Aaauggh, Aah ... 
    'u[u|g|h|m|n]+$', # ug, uh, um ..
    'Noo', 
    'Oo[h|f|o]*$', # Ooh, Oof ..
    '[O|o]+[u|i|w|l|p]*$',
    'e+m*$',
    '([H|h][a|e|h|o|y|l]+)$~[H|h]e[ll]*$', # Hello, Hallo, Holy, Hee ...
    '[S|s]h+$', # Shh
    'whoa', 
    '[Y|y]*[E|e]*$' # Yee, ye ...
]

cond = False
for exp in UT_list:
    extra = True
    if '~' in exp:
        sp = exp.split('~')
        exp = sp[0]
        extra = ~(j['words'].str.match(sp[1]))
    cond = ((j['words'].str.match(exp)) & (extra)) | (cond)
j.loc[cond, 'new_tag'] = 'UT'

In [40]:
j[cond].sample(frac=1).head()

,words,p,tag,first,sftag,entity,parsed_sftag,counts,new_tag
2049,Ha,NaN,NaN,H,FW,O,SYM,1.0,UT
36,Aaaaugh,NaN,NaN,A,NNP,PERSON,NNP,2.0,UT
52,Ah,NaN,NaN,A,NN,O,NN,2.0,UT
304,Ooof,NaN,NaN,O,NN,O,NNP,1.0,UT
457,aaaagh,NaN,NaN,a,IN,O,NN,1.0,UT


### Phase 6. Wrap up
------
1. Remove duplicates
2. Sorting
3. Counting

In [41]:
j.drop_duplicates(['words', 'new_tag'], inplace=True)

j.sort_values(['new_tag', 'words', 'counts'], ascending=False, inplace=True)

j.loc[~j['counts'].isnull(), 'counts'] += 1
j.loc[j['counts'].isnull(), 'counts'] = 1

j['group_counts'] = 1
j['group_counts'] = j.groupby(['new_tag']).transform('count')
j.loc[j['group_counts']==1, 'counts'] = 1

j['counts'] = j['counts'].astype(int)

In [42]:
j.sample(frac=1).head()

,words,p,tag,first,sftag,entity,parsed_sftag,counts,new_tag,group_counts
950,for,1.0,IN,f,IN,O,IN,16,IN,74
2115,cast,NaN,NaN,c,NN,O,VBN,2,NN,574
688,castanets,NaN,NaN,c,NNS,O,NNS,2,NNS,161
1222,live,NaN,NaN,l,VB,O,VB,3,VB,203
599,biggest,NaN,NaN,b,JJS,O,JJS,2,JJS,21


### Phase 7.  Export to file

In [43]:
j[['counts', 'new_tag', 'words']].to_csv('_Tagged_Vocab.gr', header=False, sep=" ", index=False, escapechar=' ', doublequote=False)
# Handle escape char
with open('_Tagged_Vocab.gr', 'r') as f:
    with open('Tagged_Vocab.gr', 'w+') as mf:
        text = f.readline()
        while text:
            if "  " in text:
                mf.write(text.replace("  ", " "))
                text = f.readline()
                continue
            mf.write(text)
            text = f.readline()
os.remove('_Tagged_Vocab.gr')

###  [Back to contents](#part-0)

# Part 3. Grammer Generation - S1  <a id='part-3'></a>

### Produce parsed trees with Berkeley Neural Parser

In [44]:
bst_parser = benepar.Parser("benepar_en")
with open('example_sentences.txt') as fh:
    sents = [line.strip() for line in fh]

trees = [str(bst_parser.parse(sent)) for sent in sents]
with open('example_sentences.trees', 'w') as f:
    for tree in trees:
        f.write(tree)
        f.write('\n')

### Generate rules from parsed trees

- Read parsed trees with BracketParseCorpusReader.
- Convert each tree to its chomsky normal form.
- Generate the productions (rules) that correspond to the non-terminal nodes of the tree.
- Count and save number of occurrences for each rule as weight.
- Count and save the number of occurrences for each type of sentences, e.g., S, SQ, SBARQ.
- Write the rules and manually modify the rules that generate cycle erros.
- Adjust weights for S and S2 accordingly.

In [45]:
from nltk.corpus.reader import BracketParseCorpusReader
from nltk import treetransforms

def train(training):
    """
    Generate maximum likelihood estimates of
    productions from a training file of s-expression trees.
    """
    reader = BracketParseCorpusReader('.', training)
    counts = {}
    labels = {}
    for tree in reader.parsed_sents():
        tree.chomsky_normal_form(horzMarkov=2)
        label = tree.label()
        if label not in labels:
            labels[label] = 0
        labels[label] += 1
        productions = tree.productions()
        for production in productions:
            if production not in counts:
                counts[production] = 0
            counts[production] += 1
        # productions += tree.productions()

    lhs_map = {}
    for production in counts.keys():
        if production.lhs() not in lhs_map:
            lhs_map[production.lhs()] = []
        lhs_map[production.lhs()].append(production)

    keys = lhs_map.keys()
    keys = sorted(keys)

    weights = {}
    for _, productions in lhs_map.items():
        for production in productions:
            weights[production] = counts[production]
    return weights, labels, lhs_map

def write_rules(lhs_map, labels, weights, allowed_words, grammar):
    with open(allowed_words, 'r') as f:
        lines = f.readlines()
    allowed_words = set(map(lambda l: l.strip(), lines))
    
    # print(lhs_map)
    keys = lhs_map.keys()
    keys = sorted(keys)
    with open(grammar, 'w') as f:
        # f.write('99\tTOP\tS\n')
        f.write('10\tTOP\tS2\n')
        for label, count in labels.items():
            f.write("%d\tTOP\t%s\n" % (count, label))
        for key in keys:
            productions = lhs_map[key]
            # f.write("#\n")
            for production in productions:
                valid_mask = (len(str(production.lhs())) > 0)
                rhs_symbols = [str(nt) for nt in production.rhs()]
#                 print(rhs_symbols)
                vocab_mask = (len(rhs_symbols) == 1) & (rhs_symbols[0] in allowed_words)
                if valid_mask & (not vocab_mask):                    
                    f.write("%d\t%s\t%s\n" % (weights[production],
                                              production.lhs(),
                                              ' '.join(rhs_symbols)))

Train grammar from example_sentences.trees and/or devset.trees

In [46]:
training = 'example_sentences.trees' # only for example_sentences.trees
# training = 'training.trees'  # for example_sentences.trees and devset.trees
grammar = 'S1_tunedE.gr'
allowed_words = 'allowed_words.txt'

weights, labels, lhs_map = train(training)
write_rules(lhs_map, labels, weights, allowed_words, grammar)

###  [Back to contents](#part-0)

# Part 4. Grammer Generation - S2  <a id='part-4'></a>

### Build grammars for S2

Initialize two python dictionaries. One is used to store the vocabulary. The other one is used to store the rules.<br>
The key of **vocab_dict** is each word existing in **'allowed_words.txt'**. The value of this dictionary is a list of tuples. Each tuple stores the POSTAG of the word and its counts. EXAMPLE: `'good': [('JJ', 1)]`<br>
The key of **rules_dict** is the rule. It is a python tuple. The value of this dictionary is the count of this rule. If the right hand side of the rule only contains one item, the second item is set to `'<Unary>'`. EXAMPLE: `('\_NNP', ('NNP', '\_RB')): 13`

In [49]:
rules_dict = dict()
vocab_dict = dict()

Based on **Tagged_Vocab.gr**, the **vocab_dict** is generated

In [50]:
with open('Tagged_Vocab.gr', 'r') as f_in:
    for line in f_in:
        line_arr = line.strip().split()
        token = line_arr[2]
        tag = line_arr[1]
        count = line_arr[0]
        if token not in vocab_dict.keys():
            # 'good': [('JJ', 1)]
            vocab_dict[token] = [(tag, count)]
        else:
            vocab_dict[token].append((tag, count))

Use the sentences from three files to generate rules for S2.
- **'devset.txt'**    The devset from the homework repository.
- **'example_sentences.txt'** The example sentences from the homework repository.
- **'quotes_new_preprocessed.txt'** The more sentences. See sections above for more details. 

In [61]:
with open('devset.txt') as fh:
    sents = [line.strip() for line in fh]
with open('example_sentences.txt') as fh:
    sents += [line.strip() for line in fh]
with open('quotes_new_preprocessed.txt') as fh:
    sents += [line.strip() for line in fh]

Generate the rules and store them in **rules_dict**. The approach we used basically was to look at bigrams in each sentences. For each sentence, we generated a head rule. Say the sentence `Who goes there !`. Four tokens are `(WP Who)`, `(VBZ goes)`, `(EX there)`, `(END ?)`. Five rules generated by order are `S2 -> _WP`, `_WP -> WP + _VBZ`, `_VBZ -> VBZ + _EX`, `_EX -> EX + _END`, `_END -> END`. When we stored the rules, if this rule has appeared in the keys of the **rules_dict**, the count will increment by one. Otherwise a count of one will be assigned to this rule. The rule can be used as the weight since the more we come across this rule, the more likely this rule should be used while parsing a sentence. One fact is that one token might have multiple tags. We notice this fact and decide to consider all possible rules if multiple tags are associated with the same token. In case the sentences do not end up with a punctuation, we decide to add some more extra rules to handle this situation. The rule looks like `_EX -> EX`.

In [63]:
for sent in sents:
    tokens = sent.split()
    if len(tokens) < 2:
        continue
    thisTokenTags = vocab_dict[tokens[0]]
    nextTokenTags = vocab_dict[tokens[1]]
    for thisTokenTag in thisTokenTags:
        thisTag = thisTokenTag[0]
        head_rule = ('S2', (f'_{thisTag}', '<Unary>'))
        if head_rule in rules_dict.keys():
            rules_dict[head_rule] += 1
        else:
            rules_dict[head_rule] = 1
    for i in range(len(tokens)-1):
        thisTokenTags = vocab_dict[tokens[i]]
        nextTokenTags = vocab_dict[tokens[i+1]]
        for thisTokenTag in thisTokenTags:
            for nextTokenTag in nextTokenTags:
                thisTag = thisTokenTag[0]
                nextTag = nextTokenTag[0]
                next_rule = (f'_{thisTag}', (f'{thisTag}', f'_{nextTag}'))
                if next_rule in rules_dict.keys():
                    rules_dict[next_rule] += 1
                else:
                    rules_dict[next_rule] = 1
        if i==len(tokens)-2:
            nextTokenTags = vocab_dict[tokens[i+1]]
            for nextTokenTag in nextTokenTags:
                nextTag = nextTokenTag[0]
                final_rule = (f'_{nextTag}',(f'{nextTag}','<Unary>'))
                if final_rule in rules_dict.keys():
                    rules_dict[final_rule] += 1
                else:
                    rules_dict[final_rule] = 1
            # although the sentences in devset and example always end up with a punctuation,
            # we decide to add rules like _NNP -> NNP to handle the situation of no punctuation
            thisTokenTags = vocab_dict[tokens[i]]
            for thisTokenTag in thisTokenTags:
                thisTag = thisTokenTag[0]
                extra_rule = (f'_{thisTag}',(f'{thisTag}','<Unary>'))
                if extra_rule in rules_dict.keys():
                    rules_dict[extra_rule] += 1
                else:
                    rules_dict[extra_rule] = 1

This is how **rules_dict** looks like.

In [64]:
rules_dict

{('S2', ('_NNP', '<Unary>')): 209,
 ('_NNP', ('NNP', '_RB')): 27,
 ('_NNP', ('NNP', '_EX')): 5,
 ('_RB', ('RB', '_END')): 309,
 ('_EX', ('EX', '_END')): 21,
 ('_END', ('END', '<Unary>')): 2294,
 ('_RB', ('RB', '<Unary>')): 331,
 ('_EX', ('EX', '<Unary>')): 22,
 ('S2', ('_VB', '<Unary>')): 251,
 ('_VB', ('VB', '_END')): 194,
 ('_VB', ('VB', '<Unary>')): 213,
 ('S2', ('_WP', '<Unary>')): 163,
 ('_WP', ('WP', '_VBZ')): 81,
 ('_VBZ', ('VBZ', '_RB')): 104,
 ('_VBZ', ('VBZ', '_EX')): 10,
 ('S2', ('_PRP', '<Unary>')): 516,
 ('_PRP', ('PRP', '_VBZ')): 185,
 ('_VBZ', ('VBZ', '_PRP')): 37,
 ('_PRP', ('PRP', '_PAUSE')): 62,
 ('_PAUSE', ('PAUSE', '_NNP')): 100,
 ('_NNP', ('NNP', '_PAUSE')): 149,
 ('_PAUSE', ('PAUSE', '_NN')): 123,
 ('_NN', ('NN', '_IN')): 344,
 ('_IN', ('IN', '_NNP')): 123,
 ('_NNP', ('NNP', '_NNP')): 171,
 ('_PAUSE', ('PAUSE', '_IN')): 85,
 ('_IN', ('IN', '_DT')): 432,
 ('_DT', ('DT', '_NN')): 648,
 ('_NNP', ('NNP', '_END')): 278,
 ('_NNP', ('NNP', '<Unary>')): 295,
 ('_NNP', ('N

Define a function to write the rules into a text file.

In [65]:
def output_rules(OutputFileName, count_dict):
    '''
    store the count_dict locally with correct format
    OutputFileName -- string, the name of the output file
    count_dict -- dictionary, contains the updated count
    '''    
    f_output = open(OutputFileName, "w")
    f_output.write(f'80\tTOP\tS2\n')
    for key, value in count_dict.items():
        if key[1][1] == '<Unary>':
            f_output.write(f'{value}\t{key[0]}\t{key[1][0]}\n')
        else:
            f_output.write(f'{value}\t{key[0]}\t{key[1][0]} {key[1][1]}\n')
    f_output.close()

Call the function defined above to actually store the grammers.

In [66]:
output_rules('S2_test.gr', rules_dict)

Here we combine S1 grammars and S2 grammars. The **'Tagged_Vocab.gr'** is used as a vocabulary file. Out of 100, we put 20 on S1 and 80 on S2. These two values are not chosen because they produced better cross entropy on both sets of sentences. The reason we chose this ratio is because we believe S2 is able to cover much much more sentences than S1. And we didn't lose too much cross entropy even the weight of S2 becomes larger. To work better on the sentences generated from other groups, we finally decide this ratio as is. Below is how the grammar works on three datasets. For the actual output, see [Part 5](#part-5)
- On the **'example_sentences.txt'** file, the cross entropy is -7.68701.
- On the **'devset.txt'** file, the cross entropy is -8.24257.
- On the extra sentences we preprocessed, the cross entropy is -7.72603.

In order to parse any random sentence, we generate all positive rules. The approach is to go over any combination of two POSTAGs to generate a rule.

In [ ]:
extra_dict = {1970: 'WRB',  1969: 'WP$',
  1967: 'WP',  1963: 'WDT',  1981: 'VBZ',  1935: 'VBP',
  2008: 'VBN',  2284: 'VBG',  1999: 'VBD',  2013: 'VB',
  2012: 'UT',  2018: 'UH',  1834: 'TO',  1955: 'SYM',
  1892: 'RP',  1314: 'RBS',  1311: 'RBR',  2025: 'RB',
  455: 'QUOTE',  2023: 'PRP$',  2024: 'PRP',  12: 'POS',
  489: 'PDT',  16: 'PAUSE',  2000: 'NNS',  244: 'NNPS',
  1052: 'NNP',  2028: 'NN',  2006: 'MD',  1103: 'LS',
  2004: 'JJS',  2003: 'JJR',  2021: 'JJ',  1987: 'IN',
  2143: 'FW',  1799: 'EX',  15: 'ENDBK',  32: 'END',
  1812: 'DT',  1867: 'CD',  1397: 'CC',  31: 'BREAK',
  14: 'BGNBK'}

fh = open("S2.gr", "w")
fh.write(f'1\tTOP\tS3\n')
for item_1 in extra_dict.values():
    fh.write(f'1\tS3\t_{item_1}\n')
    for item_2 in extra_dict.values():
        this_rule = (f'_{item_1}', (f'{item_1}', f'_{item_2}'))
        if this_rule not in rules_dict:
            fh.write(f'1\t_{item_1}\t{item_1} _{item_2}\n')
fh.close()

###  [Back to contents](#part-0)

# Part 5. Parsing sentances  <a id='part-5'></a>

In [14]:
%%time
with open('example_sentences.txt') as fh:
    sents = [line.strip() for line in fh]
parse_gram = Pcfg(["S1_tunedE.gr" , "S2_test.gr", "Tagged_Vocab.gr"])
parser = CkyParse(parse_gram, beamsize=0.0001)
ce, trees = parser.parse_sentences(sents)
print("-cross entropy: {}".format(ce))

(TOP (S (NP (NNP Arthur) ) (S|<VP-END> (VP (VBZ is) (NP (DT the) (NN king))) (END .))) )
(TOP (S2 (_NNP (NNP Arthur) (_NNS (NNS rides) (_DT (DT the) (_NN (NN horse) (_IN (IN near) (_DT (DT the) (_NN (NN castle) (_END (END .) )))))))) ) )
(TOP (S2 (_VBG (VBG riding) (_TO (TO to) (_NNP (NNP Camelot) (_VBZ (VBZ is) (_RB (RB hard) (_END (END .) )))))) ) )
(TOP (SQ (VBP do) (SQ|<NP-VP> (NP (NNS coconuts) ) (SQ|<VP-END> (VP (VB speak) ) (END ?)))) )
(TOP (SBARQ (WHNP (WP what) ) (SBARQ|<SQ-END> (SQ (VBZ does) (SQ|<NP-VP> (NP (NNP Arthur) ) (VP (VB ride) ))) (END ?))) )
(TOP (SBARQ (WHNP (WP who) ) (SBARQ|<SQ-END> (SQ (VBZ does) (SQ|<NP-VP> (NP (NNP Arthur) ) (VP (VB suggest) (SBAR (S (NP (PRP she) ) (VP (VB carry) )) )))) (END ?))) )
(TOP (SBARQ (WHADVP (WRB why) ) (SBARQ|<SQ-END> (SQ (VBZ does) (SQ|<NP-VP> (NP (NNP England) ) (VP (VB have) (NP (DT a) (NN king))))) (END ?))) )
(TOP (SQ (VBP are) (SQ|<NP-VP> (NP (PRP they) ) (VP (VBG suggesting) (SBAR (S (NP (NNP Arthur) ) (S|<VP-END> (VP (NN

#reading grammar file: S1_tunedE.gr
#reading grammar file: S2_test.gr
#reading grammar file: Tagged_Vocab.gr
#parsing: ['Arthur', 'is', 'the', 'king', '.']
#parsing: ['Arthur', 'rides', 'the', 'horse', 'near', 'the', 'castle', '.']
#parsing: ['riding', 'to', 'Camelot', 'is', 'hard', '.']
#parsing: ['do', 'coconuts', 'speak', '?']
#parsing: ['what', 'does', 'Arthur', 'ride', '?']
#parsing: ['who', 'does', 'Arthur', 'suggest', 'she', 'carry', '?']
#parsing: ['why', 'does', 'England', 'have', 'a', 'king', '?']
#parsing: ['are', 'they', 'suggesting', 'Arthur', 'ride', 'to', 'Camelot', '?']
#parsing: ['five', 'strangers', 'are', 'at', 'the', 'Round', 'Table', '.']
#parsing: ['Guinevere', 'might', 'have', 'known', '.']
#parsing: ['Guinevere', 'should', 'be', 'riding', 'with', 'Patsy', '.']
#parsing: ['it', 'is', 'Sir', 'Lancelot', 'who', 'knows', 'Zoot', '!']
#parsing: ['either', 'Arthur', 'knows', 'or', 'Patsy', 'does', '.']
#parsing: ['neither', 'Sir', 'Lancelot', 'nor', 'Guinevere', 'will

In [27]:
%%time
with open('devset.txt') as fh:
    sents = [line.strip() for line in fh]
parse_gram = Pcfg(["S1_tunedE.gr" , "S2_test.gr", "Tagged_Vocab.gr"])
parser = CkyParse(parse_gram, beamsize=0.0001)
ce, trees = parser.parse_sentences(sents)
print("-cross entropy: {}".format(ce))

#reading grammar file: S1_tunedE.gr
#reading grammar file: S2_test.gr
#reading grammar file: Tagged_Vocab.gr
#parsing: ['Whoa', 'there', '!']
#parsing: ['Halt', '!']
#parsing: ['Who', 'goes', 'there', '?']
#parsing: ['It', 'is', 'I', ',', 'Arthur', ',', 'son', 'of', 'Uther', 'Pendragon', ',', 'from', 'the', 'castle', 'of', 'Camelot', '.']
#parsing: ['King', 'of', 'the', 'Britons', ',', 'defeator', 'of', 'the', 'Saxons', ',', 'sovereign', 'of', 'all', 'England', '!']
#parsing: ['Pull', 'the', 'other', 'one', '!']
#parsing: ['I', 'am', '.']
#parsing: ['And', 'this', 'my', 'trusty', 'servant', 'Patsy', '.']
#parsing: ['We', 'have', 'ridden', 'the', 'length', 'and', 'breadth', 'of', 'the', 'land', 'in', 'search', 'of', 'knights', 'who', 'will', 'join', 'me', 'in', 'my', 'court', 'of', 'Camelot', '.']
#parsing: ['I', 'must', 'speak', 'with', 'your', 'lord', 'and', 'master', '.']
#parsing: ['What', ',', 'ridden', 'on', 'a', 'horse', '?']
#parsing: ['Yes', '!']
#parsing: ["You're", 'using', '

(TOP (S2 (_NNP (NNP Whoa) (_EX (EX there) (_END (END !) ))) ) )
(TOP (S2 (_VB (VB Halt) (_END (END !) )) ) )
(TOP (S2 (_WP (WP Who) (_VBZ (VBZ goes) (_EX (EX there) (_END (END ?) )))) ) )
(TOP (S2 (_PRP (PRP It) (_VBZ (VBZ is) (_PRP (PRP I) (_PAUSE (PAUSE ,) (_NNP (NNP Arthur) (_PAUSE (PAUSE ,) (_NN (NN son) (_IN (IN of) (_NNP (NNP Uther) (_NNP (NNP Pendragon) (_PAUSE (PAUSE ,) (_IN (IN from) (_DT (DT the) (_NN (NN castle) (_IN (IN of) (_NNP (NNP Camelot) (_END (END .) ))))))))))))))))) ) )
(TOP (S2 (_NNP (NNP King) (_IN (IN of) (_DT (DT the) (_NNPS (NNPS Britons) (_PAUSE (PAUSE ,) (_NN (NN defeator) (_IN (IN of) (_DT (DT the) (_NNS (NNS Saxons) (_PAUSE (PAUSE ,) (_JJ (JJ sovereign) (_IN (IN of) (_DT (DT all) (_NNP (NNP England) (_END (END !) ))))))))))))))) ) )
(TOP (S2 (_VB (VB Pull) (_DT (DT the) (_JJ (JJ other) (_CD (CD one) (_END (END !) ))))) ) )
(TOP (S2 (_PRP (PRP I) (_VBP (VBP am) (_END (END .) ))) ) )
(TOP (S2 (_CC (CC And) (_DT (DT this) (_PRP$ (PRP$ my) (_JJ (JJ trusty) (_J

#parsing: ['If', 'there', "'s", 'ever', 'going', 'to', 'be', 'any', 'progress', '--']
#parsing: ['Dennis', ',', 'there', "'s", 'some', 'lovely', 'filth', 'down', 'here', '.']
#parsing: ['Oh', '--', 'how', "d'", 'you', 'do', '?']
#parsing: ['How', 'do', 'you', 'do', ',', 'good', 'lady', '.']
#parsing: ['I', 'am', 'Arthur', ',', 'King', 'of', 'the', 'Britons', '.']
#parsing: ['Who', "'s", 'castle', 'is', 'that', '?']
#parsing: ['King', 'of', 'the', 'who', '?']
#parsing: ['The', 'Britons', '.']
#parsing: ['Who', 'are', 'the', 'Britons', '?']
#parsing: ['Well', ',', 'we', 'all', 'are', '.']
#parsing: ["we're", 'all', 'Britons', 'and', 'I', 'am', 'your', 'king', '.']
#parsing: ['I', 'did', "n't", 'know', 'we', 'had', 'a', 'king', '.']
#parsing: ['I', 'thought', 'we', 'were', 'an', 'autonomous', 'collective', '.']
#parsing: ["You're", 'fooling', 'yourself', '.']
#parsing: ["We're", 'living', 'in', 'a', 'dictatorship', '.']
#parsing: ['A', 'self', '-', 'perpetuating', 'autocracy', 'in', 'whic

(TOP (S2 (_IN (IN By) (_NN (NN exploitin) (_QUOTE (QUOTE ') (_DT (DT the) (_NNS (NNS workers) (_BREAK (BREAK --) (_IN (IN by) (_QUOTE (QUOTE ') (_NN (NN angin) (_QUOTE (QUOTE ') (_IN (IN on) (_TO (TO to) (_VBN (VBN outdated) (_NN (NN imperialist) (_NN (NN dogma) (_WDT (WDT which) (_VBZ (VBZ perpetuates) (_DT (DT the) (_JJ (JJ economic) (_DT (DT an) (_QUOTE (QUOTE ') (_JJ (JJ social) (_NNS (NNS differences) (_IN (IN in) (_PRP$ (PRP$ our) (_NN (NN society) (_END (END !) ))))))))))))))))))))))))))) ) )
(TOP (S2 (_IN (IN If) (_EX (EX there) (_POS (POS 's) (_RB (RB ever) (_VBG (VBG going) (_TO (TO to) (_VB (VB be) (_DT (DT any) (_NN (NN progress) (_BREAK (BREAK --) )))))))))) ) )
(TOP (S2 (_NNP (NNP Dennis) (_PAUSE (PAUSE ,) (_EX (EX there) (_POS (POS 's) (_DT (DT some) (_JJ (JJ lovely) (_NN (NN filth) (_RP (RP down) (_RB (RB here) (_END (END .) )))))))))) ) )
(TOP (S2 (_UH (UH Oh) (_BREAK (BREAK --) (_WRB (WRB how) (_IN (IN d') (_PRP (PRP you) (_VBP (VBP do) (_END (END ?) ))))))) ) )
(TOP 

#parsing: ['The', 'pond', '!']
#parsing: ['What', 'also', 'floats', 'in', 'water', '?']
#parsing: ['Bread', '!']
#parsing: ['Apples', '!']
#parsing: ['Very', 'small', 'rocks', '!']
#parsing: ['Cider', '!']
#parsing: ['Great', 'gravy', '!']
#parsing: ['Cherries', '!']
#parsing: ['Mud', '!']
#parsing: ['Churches', '--', 'churches', '!']
#parsing: ['Lead', '--', 'lead', '!']
#parsing: ['A', 'duck', '.']
#parsing: ['Oooh', '.']
#parsing: ['Exactly', '!']
#parsing: ['So', ',', 'logically', '...', ',']
#parsing: ['If', '...']
#parsing: ['she.', '.']
#parsing: ['weighs', 'the', 'same', 'as', 'a', 'duck', ',', 'she', "'s", 'made', 'of', 'wood', '.']
#parsing: ['And', 'therefore', '--', '?']
#parsing: ['A', 'witch', '!']
#parsing: ['We', 'shall', 'use', 'my', 'larger', 'scales', '!']
#parsing: ['Right', ',', 'remove', 'the', 'supports', '!']
#parsing: ['A', 'witch', '!']
#parsing: ['It', "'s", 'a', 'fair', 'cop', '.']
#parsing: ['Burn', 'her', '!']
#parsing: ['Burn', '!']
#parsing: ['Who', 'are


(TOP (S2 (_DT (DT The) (_NN (NN pond) (_END (END !) ))) ) )
(TOP (S2 (_WDT (WDT What) (_RB (RB also) (_VBZ (VBZ floats) (_IN (IN in) (_NN (NN water) (_END (END ?) )))))) ) )
(TOP (S2 (_NN (NN Bread) (_END (END !) )) ) )
(TOP (S2 (_NNS (NNS Apples) (_END (END !) )) ) )
(TOP (S2 (_RB (RB Very) (_JJ (JJ small) (_NNS (NNS rocks) (_END (END !) )))) ) )
(TOP (S2 (_NNP (NNP Cider) (_END (END !) )) ) )
(TOP (S2 (_NNP (NNP Great) (_NN (NN gravy) (_END (END !) ))) ) )
(TOP (S2 (_NNS (NNS Cherries) (_END (END !) )) ) )
(TOP (S2 (_NN (NN Mud) (_END (END !) )) ) )
(TOP (S2 (_NNS (NNS Churches) (_BREAK (BREAK --) (_NNS (NNS churches) (_END (END !) )))) ) )
(TOP (S2 (_JJ (JJ Lead) (_BREAK (BREAK --) (_NN (NN lead) (_END (END !) )))) ) )
(TOP (S2 (_DT (DT A) (_NN (NN duck) (_END (END .) ))) ) )
(TOP (S2 (_UT (UT Oooh) (_END (END .) )) ) )
(TOP (S2 (_RB (RB Exactly) (_END (END !) )) ) )
(TOP (S2 (_RB (RB So) (_PAUSE (PAUSE ,) (_RB (RB logically) (_BREAK (BREAK ...) (_PAUSE (PAUSE ,) ))))) ) )
(TOP (S2

#parsing: ['Together', 'they', 'formed', 'a', 'band', 'whose', 'names', 'and', 'deeds', 'were', 'to', 'be', 'retold', 'throughout', 'the', 'centuries', ',', 'the', 'Knights', 'of', 'the', 'Round', 'Table', '.']
#parsing: ['And', 'that', ',', 'my', 'liege', ',', 'is', 'how', 'we', 'know', 'the', 'Earth', 'to', 'be', 'banana', '-', 'shaped', '.']
#parsing: ['This', 'new', 'learning', 'amazes', 'me', ',', 'Sir', 'Bedemir', '.']
#parsing: ['Explain', 'again', 'how', "sheeps'", 'bladders', 'may', 'be', 'employed', 'to', 'prevent', 'earthquakes', '.']
#parsing: ['Oh', ',', 'certainly', ',', 'sir', '.']
#parsing: ['Look', ',', 'my', 'liege', '!']
#parsing: ['Camelot', '!']
#parsing: ['It', "'s", 'only', 'a', 'model', '.']
#parsing: ['Shhh', '!']
#parsing: ['Knights', ',', 'I', 'bid', 'you', 'welcome', 'to', 'your', 'new', 'home', '.']
#parsing: ['Let', 'us', 'ride', '...']
#parsing: ['to', 'Camelot', '.']
#parsing: ["We're", 'knights', 'of', 'the', 'round', 'table']
#parsing: ['We', 'dance', 


(TOP (S2 (_UH (UH No) (_PAUSE (PAUSE ,) (_RB (RB now) (_VB (VB go) (_RB (RB away) (_CC (CC or) (_PRP (PRP I) (_MD (MD shall) (_VB (VB taunt) (_PRP (PRP you) (_DT (DT a) (_JJ (JJ second) (_NN (NN time) (_BREAK (BREAK -) (_DT (DT a) (_END (END !) )))))))))))))))) ) )
(TOP (S2 (_RB (RB Now) (_PAUSE (PAUSE ,) (_DT (DT this) (_VBZ (VBZ is) (_PRP$ (PRP$ your) (_JJ (JJ last) (_NN (NN chance) (_END (END .) )))))))) ) )
(TOP (S2 (_PRP (PRP I've) (_VBN (VBN been) (_RBR (RBR more) (_IN (IN than) (_JJ (JJ reasonable) (_END (END .) )))))) ) )
(TOP (S2 (_BGNBK (BGNBK () (_NNP (NNP Fetchez) (_DT (DT la) (_NN (NN vache) (_END (END .) (_ENDBK (ENDBK )) (_NN (NN wha) (_END (END ?) )))))))) ) )
(TOP (S2 (_BGNBK (BGNBK () (_NNP (NNP Fetchez) (_DT (DT la) (_NN (NN vache) (_END (END !) (_ENDBK (ENDBK )) )))))) ) )
(TOP (S2 (_IN (IN If) (_PRP (PRP you) (_VBP (VBP do) (_RB (RB not) (_VB (VB agree) (_TO (TO to) (_PRP$ (PRP$ my) (_NNS (NNS commands) (_PAUSE (PAUSE ,) (_RB (RB then) (_PRP (PRP I) (_MD (MD shall

(TOP (S2 (_IN (IN In) (_DT (DT that) (_NN (NN case) (_PRP (PRP I) (_MD (MD shall) (_VBP (VBP have) (_TO (TO to) (_VB (VB kill) (_PRP (PRP you) (_END (END .) )))))))))) ) )
(TOP (S2 (_NNP (NNP Shall) (_PRP (PRP I) (_END (END ?) ))) ) )
(TOP (S2 (_UH (UH Oh) (_PAUSE (PAUSE ,) (_PRP (PRP I) (_VBP (VBP do) (_RB (RB n't) (_VBP (VBP think) (_RB (RB so) (_END (END .) )))))))) ) )
(TOP (S2 (_RB (RB Well) (_PAUSE (PAUSE ,) (_WP (WP what) (_VBP (VBP do) (_PRP (PRP I) (_VBP (VBP think) (_END (END ?) ))))))) ) )
(TOP (S2 (_PRP (PRP I) (_VBP (VBP think) (_VB (VB kill) (_PRP (PRP him) (_END (END .) ))))) ) )
(TOP (S2 (_RB (RB Well) (_VB (VB let) (_POS (POS 's) (_VB (VB be) (_JJ (JJ nice) (_TO (TO to) (_PRP (PRP him) (_END (END .) )))))))) ) )
(TOP (S2 (_UH (UH Oh) (_VBD (VBD shut) (_RP (RP up) (_END (END .) )))) ) )
(TOP (S2 (_RB (RB Perhaps) (_BREAK (BREAK -) )) ) )
(TOP (S2 (_CC (CC And) (_PRP (PRP you) (_END (END .) ))) ) )
(TOP (S2 (_UH (UH Oh) (_JJ (JJ quick) (_VB (VB get) (_DT (DT the) (_NN (N

#parsing: ['No', ',', 'now', 'go', 'away', 'or', 'I', 'shall', 'taunt', 'you', 'a', 'second', 'time', '-', 'a', '!']
#parsing: ['Now', ',', 'this', 'is', 'your', 'last', 'chance', '.']
#parsing: ["I've", 'been', 'more', 'than', 'reasonable', '.']
#parsing: ['(', 'Fetchez', 'la', 'vache', '.', ')', 'wha', '?']
#parsing: ['(', 'Fetchez', 'la', 'vache', '!', ')']
#parsing: ['If', 'you', 'do', 'not', 'agree', 'to', 'my', 'commands', ',', 'then', 'I', 'shall', '--']
#parsing: ['Jesus', 'Christ', '!']
#parsing: ['Right', '!']
#parsing: ['Charge', '!']
#parsing: ['Ah', ',', 'this', 'one', 'is', 'for', 'your', 'mother', '!']
#parsing: ['Run', 'away', '!']
#parsing: ['Thpppt', '!']
#parsing: ['Fiends', '!']
#parsing: ['I', "'ll", 'tear', 'them', 'apart', '!']
#parsing: ['No', 'no', ',', 'no', '.']
#parsing: ['Sir', '!']
#parsing: ['I', 'have', 'a', 'plan', ',', 'sir', '.']
#parsing: ['ce', 'labon', 'a', 'bunny', 'do', 'wha', '?']
#parsing: ['un', 'cadeau', '?']
#parsing: ['a', 'present', '!']
#

In [28]:
%%time
with open('quotes_new_preprocessed.txt') as fh:
    sents = [line.strip() for line in fh]
parse_gram = Pcfg(["S1_tunedE.gr" , "S2_test.gr", "Tagged_Vocab.gr"])
parser = CkyParse(parse_gram, beamsize=0.0001)
ce, trees = parser.parse_sentences(sents)
print("-cross entropy: {}".format(ce))

#reading grammar file: S1_tunedE.gr
#reading grammar file: S2_test.gr
#reading grammar file: Tagged_Vocab.gr
#parsing: ['All', 'right', ',', 'all', 'right', ',', 'not', 'biscuits', ',', 'but', 'lets', 'kill', 'him', 'anyway', '.']
#parsing: ['Right', '!']
#parsing: ['He', 'buggered', 'off', '.']
#parsing: ['So', 'he', 'has', ',', 'he', "'s", 'scarpered', '.']
#parsing: ['Brave', 'Sir', 'Robin', 'ran', 'away', 'No', '!']
#parsing: ['Bravely', 'ran', 'away', 'away', 'I', 'did', "n't", '!']
#parsing: ['When', 'danger', 'reared', 'its', 'ugly', 'head', ',', 'He', 'bravely', 'turned', 'his', 'tail', 'and', 'fled', 'No', '!']
#parsing: ['Yes', 'Brave', 'Sir', 'Robin', 'turned', 'about', 'I', 'did', "n't", '!']
#parsing: ['And', 'gallantly', 'he', 'chickened', 'out', 'Bravely', 'taking', 'to', 'his', 'feet', 'I', 'never', 'did', '!']
#parsing: ['He', 'beat', 'a', 'very', 'brave', 'retreat', 'Oh', ',', 'lie', '!']
#parsing: ['Bravest', 'of', 'the', 'brave', 'Sir', 'Robin', 'I', 'never', '!']
#

(TOP (S2 (_PDT (PDT All) (_RB (RB right) (_PAUSE (PAUSE ,) (_PDT (PDT all) (_RB (RB right) (_PAUSE (PAUSE ,) (_RB (RB not) (_NNS (NNS biscuits) (_PAUSE (PAUSE ,) (_CC (CC but) (_VBZ (VBZ lets) (_VB (VB kill) (_PRP (PRP him) (_RB (RB anyway) (_END (END .) ))))))))))))))) ) )
(TOP (S2 (_RB (RB Right) (_END (END !) )) ) )
(TOP (S2 (_PRP (PRP He) (_VBD (VBD buggered) (_RP (RP off) (_END (END .) )))) ) )
(TOP (S2 (_RB (RB So) (_PRP (PRP he) (_VBZ (VBZ has) (_PAUSE (PAUSE ,) (_PRP (PRP he) (_POS (POS 's) (_VBN (VBN scarpered) (_END (END .) )))))))) ) )
(TOP (S2 (_NNP (NNP Brave) (_NNP (NNP Sir) (_NNP (NNP Robin) (_VBD (VBD ran) (_RB (RB away) (_DT (DT No) (_END (END !) ))))))) ) )
(TOP (S2 (_RB (RB Bravely) (_VBD (VBD ran) (_RB (RB away) (_RB (RB away) (_PRP (PRP I) (_VBD (VBD did) (_RB (RB n't) (_END (END !) )))))))) ) )
(TOP (S2 (_WRB (WRB When) (_NN (NN danger) (_VBD (VBD reared) (_PRP$ (PRP$ its) (_JJ (JJ ugly) (_NN (NN head) (_PAUSE (PAUSE ,) (_PRP (PRP He) (_RB (RB bravely) (_VBD (VBD 

#parsing: ['A', 'spanking', '!']
#parsing: ['You', 'must', 'spank', 'her', 'well', '.']
#parsing: ['And', 'after', 'you', 'have', 'spanked', 'her', ',', 'you', 'may', 'deal', 'with', 'her', 'as', 'you', 'like', '.']
#parsing: ['And', 'then', ',', 'spank', 'me', '.']
#parsing: ['And', 'spank', 'me', '.']
#parsing: ['And', 'me', '.']
#parsing: ['Yes', ',', 'yes', ',', 'you', 'must', 'give', 'us', 'all', 'a', 'good', 'spanking', '!']
#parsing: ['A', 'spanking', '!']
#parsing: ['And', 'after', 'the', 'spanking', ',', 'the', 'oral', 'sex', '.']
#parsing: ['Oral', 'sex', '!']
#parsing: ['Well', ',', 'I', 'could', 'stay', 'a', 'BIT', 'longer', '.']
#parsing: ['Sir', 'Galahad', '!']
#parsing: ['Oh', ',', 'hello', '.']
#parsing: ['Quick', '!']
#parsing: ['What', '?']
#parsing: ['Quick', '!']
#parsing: ['Why', '?']
#parsing: ["You're", 'in', 'great', 'peril', '!']
#parsing: ['Silence', ',', 'foul', 'temptress', '!']
#parsing: ['Now', 'look', ',', 'it', "'s", 'not', 'important', '.']
#parsing: ['


(TOP (S2 (_UH (UH Yes) (_PAUSE (PAUSE ,) (_UH (UH yes) (_END (END .) )))) ) )
(TOP (S2 (_UH (UH Oh) (_PAUSE (PAUSE ,) (_NN (NN shit) (_END (END .) )))) ) )
(TOP (S2 (_PRP (PRP We) (_VBD (VBD were) (_IN (IN in) (_DT (DT the) (_NN (NN nick) (_IN (IN of) (_NN (NN time) (_PRP (PRP you) (_VBD (VBD were) (_IN (IN in) (_JJ (JJ great) (_NN (NN peril) (_END (END .) ))))))))))))) ) )
(TOP (S2 (_PRP (PRP I) (_VBP (VBP do) (_RB (RB n't) (_VBP (VBP think) (_PRP (PRP I) (_VBD (VBD was) (_END (END .) ))))))) ) )
(TOP (S2 (_UH (UH Yes) (_PRP (PRP you) (_VBD (VBD were) (_PAUSE (PAUSE ,) (_PRP (PRP you) (_VBD (VBD were) (_IN (IN in) (_JJ (JJ terrible) (_NN (NN peril) (_END (END .) )))))))))) ) )
(TOP (S2 (_VB (VB Look) (_PAUSE (PAUSE ,) (_VB (VB let) (_PRP (PRP me) (_VBP (VBP go) (_RB (RB back) (_IN (IN in) (_EX (EX there) (_CC (CC and) (_VBP (VBP face) (_DT (DT the) (_NN (NN peril) (_END (END .) ))))))))))))) ) )
(TOP (S2 (_UH (UH No) (_PAUSE (PAUSE ,) (_PRP (PRP it) (_POS (POS 's) (_RB (RB too) (_JJ 

#parsing: ['But', 'Father', ',', 'I', 'do', "n't", 'want', 'any', 'of', 'that', '.']
#parsing: ['Listen', ',', 'lad', '.']
#parsing: ["I've", 'built', 'this', 'kingdom', 'up', 'from', 'nothing', '.']
#parsing: ['When', 'I', 'started', 'here', ',', 'all', 'there', 'was', 'was', 'swamp', '.']
#parsing: ['All', 'the', 'kings', 'said', 'I', 'was', 'daft', 'to', 'build', 'a', 'castle', 'in', 'a', 'swamp', ',', 'but', 'I', 'built', 'it', 'all', 'the', 'same', ',', 'just', 'to', 'show', "'em", '.']
#parsing: ['It', 'sank', 'into', 'the', 'swamp', '.']
#parsing: ['So', ',', 'I', 'built', 'a', 'second', 'one', '.']
#parsing: ['That', 'sank', 'into', 'the', 'swamp', '.']
#parsing: ['So', 'I', 'built', 'a', 'third', 'one', '.']
#parsing: ['That', 'burned', 'down', ',', 'fell', 'over', ',', 'then', 'sank', 'into', 'the', 'swamp', '.']
#parsing: ['But', 'the', 'fourth', 'one', 'stayed', 'up', '.']
#parsing: ['An', "'", 'that', "'s", 'what', 'your', 'gonna', 'get', ',', 'lad', '--', 'the', 'stronges

...

(TOP (S2 (_TO (TO To) (_DT (DT the) (_RB (RB north) (_EX (EX there) (_VBZ (VBZ lies) (_DT (DT a) (_NN (NN cave) (_BREAK (BREAK --) (_DT (DT the) (_NN (NN cave) (_IN (IN of) (_NNP (NNP Kyre) (_NNP (NNP Banorg) (_BREAK (BREAK --) (_WRB (WRB wherein) (_PAUSE (PAUSE ,) (_VBN (VBN carved) (_IN (IN in) (_JJ (JJ mystic) (_NNS (NNS runes) (_IN (IN upon) (_DT (DT the) (_RB (RB very) (_VBG (VBG living) (_NN (NN rock) (_PAUSE (PAUSE ,) (_DT (DT the) (_JJ (JJ last) (_NNS (NNS words) (_IN (IN of) (_NNP (NNP Ulfin) (_NNP (NNP Bedweer) (_IN (IN of) (_NNP (NNP Regett) (_VB (VB proclaim) (_DT (DT the) (_JJ (JJ last) (_VBG (VBG resting) (_NN (NN place) (_IN (IN of) (_DT (DT the) (_RBS (RBS most) (_UT (UT Holy) (_NNP (NNP Grail) (_END (END .) ))))))))))))))))))))))))))))))))))))))))))))) ) )
(TOP (S2 (_WRB (WRB Where) (_MD (MD could) (_PRP (PRP we) (_VB (VB find) (_DT (DT this) (_NN (NN cave) (_PAUSE (PAUSE ,) (_UT (UT O) (_NNP (NNP Tim) (_DT (DT the) (_NN (NN Enchanter) (_END (END ?) )))))))))))) ) )
(T

#parsing: ['You', 'tit', '!']
#parsing: ['I', 'soiled', 'my', 'armor', 'I', 'was', 'so', 'scared', '!']
#parsing: ['Look', ',', 'that', 'rabbit', "'s", 'got', 'a', 'vicious', 'streak', 'a', 'mile', 'wide', ',', 'it', "'s", 'a', 'killer', '!']
#parsing: ['Get', 'stuffed', '!']
#parsing: ['It', "'ll", 'do', 'you', 'a', 'trick', ',', 'mate', '!']
#parsing: ['Oh', ',', 'yeah', '?']
#parsing: ['You', 'mangy', 'Scot', 'git', '!']
#parsing: ["I'm", 'warning', 'you', '!']
#parsing: ['What', "'s", 'he', 'do', ',', 'nibble', 'your', 'bum', '?']
#parsing: ['He', "'s", 'got', 'huge', ',', 'sharp', '--', 'he', 'can', 'leap', 'about', '--', 'look', 'at', 'the', 'bones', '!']
#parsing: ['Go', 'on', ',', 'Boris', '.']
#parsing: ['Chop', 'his', 'head', 'off', '!']
#parsing: ['Right', '!']
#parsing: ['Silly', 'little', 'bleeder', '.']
#parsing: ['One', 'rabbit', 'stew', 'comin', "'", 'right', 'up', '!']
#parsing: ['Look', '!']
#parsing: ['Aaaugh', '!']
#parsing: ['Jesus', 'Christ', '!']
#parsing: ['I', 

(TOP (S2 (_RB (RB Once) (_DT (DT the) (_NN (NN number) (_CD (CD three) (_PAUSE (PAUSE ,) (_VBG (VBG being) (_DT (DT the) (_JJ (JJ third) (_NN (NN number) (_PAUSE (PAUSE ,) (_VB (VB be) (_VBN (VBN reached) (_PAUSE (PAUSE ,) (_RB (RB then) (_IN (IN lobbest) (_PRP (PRP thou) (_PRP$ (PRP$ thy) (_UT (UT Holy) (_VB (VB Hand) (_NNP (NNP Grenade) (_IN (IN of) (_NNP (NNP Antioch) (_IN (IN towards) (_PRP (PRP thou) (_NN (NN foe) (_PAUSE (PAUSE ,) (_WP (WP who) (_VBG (VBG being) (_JJ (JJ naughty) (_IN (IN in) (_PRP$ (PRP$ my) (_NN (NN sight) (_PAUSE (PAUSE ,) (_MD (MD shall) (_VB (VB snuff) (_PRP (PRP it) (_END (END .) ))))))))))))))))))))))))))))))))))))) ) )
(TOP (S2 (_QUOTE (QUOTE ') (_QUOTE (QUOTE ") (_UH (UH Amen) (_END (END .) )))) ) )
(TOP (S2 (_UH (UH Amen) (_END (END .) )) ) )
(TOP (S2 (_RB (RB Right) (_END (END !) )) ) )
(TOP (S2 (_CD (CD One) (_BREAK (BREAK ...) )) ) )
(TOP (S2 (_CD (CD two) (_BREAK (BREAK ...) )) ) )
(TOP (S2 (_CD (CD five) (_END (END !) )) ) )
(TOP (S2 (_CD (CD Three

(TOP (S2 (_IN (IN In) (_DT (DT the) (_NN (NN name) (_IN (IN of) (_DT (DT the) (_NNP (NNP Lord) (_PAUSE (PAUSE ,) (_PRP (PRP we) (_VBP (VBP demand) (_NN (NN entrance) (_TO (TO to) (_DT (DT this) (_JJ (JJ sacred) (_NN (NN castle) (_END (END !) ))))))))))))))) ) )
(TOP (S2 (_DT (DT No) (_NN (NN chance) (_PAUSE (PAUSE ,) (_NNP (NNP English) (_VBG (VBG bedwetting) (_NNS (NNS types) (_END (END .) ))))))) ) )
(TOP (S2 (_PRP (PRP I) (_VBP (VBP burst) (_PRP$ (PRP$ my) (_NNS (NNS pimples) (_IN (IN at) (_PRP (PRP you) (_CC (CC and) (_VB (VB call) (_PRP$ (PRP$ your) (_NN (NN daughter) (_DT (DT an) (_JJ (JJ unrequested) (_JJ (JJ silly) (_NN (NN thing) (_END (END .) ))))))))))))))) ) )
(TOP (S2 (_PRP (PRP You) (_JJ (JJ tiny) (_BREAK (BREAK -) (_JJ (JJ brained) (_NNS (NNS wipers) (_IN (IN of) (_JJ (JJ other) (_NNS (NNS people) (_POS (POS 's) (_NNS (NNS bottoms) (_END (END !) ))))))))))) ) )
(TOP (S2 (_IN (IN If) (_PRP (PRP you) (_VBP (VBP do) (_RB (RB not) (_VB (VB open) (_DT (DT this) (_NN (NN door)

#parsing: ['The', 'cartoon', 'peril', 'was', 'no', 'more', '.']
#parsing: ['The', 'Quest', 'for', 'the', 'Holy', 'Grail', 'could', 'continue', '.']
#parsing: ['There', 'it', 'is', '!']
#parsing: ['The', 'Bridge', 'of', 'Death', '!']
#parsing: ['Oh', ',', 'great', '.']
#parsing: ['Look', '!']
#parsing: ['There', "'s", 'the', 'keeper', 'from', 'Scene', '24', '!']
#parsing: ['What', 'is', 'he', 'doing', 'here', '?']
#parsing: ['He', 'is', 'the', 'keeper', 'of', 'the', 'Bridge', 'of', 'Death', '.']
#parsing: ['He', 'asks', 'each', 'traveller', 'five', 'questions', '--', 'Three', 'questions', '.']
#parsing: ['Three', 'questions', '.']
#parsing: ['He', 'who', 'answers', 'the', 'five', 'questions', '--', 'Three', 'questions', '.']
#parsing: ['Three', 'questions', 'may', 'cross', 'in', 'safety', '.']
#parsing: ['What', 'if', 'you', 'get', 'a', 'question', 'wrong', '?']
#parsing: ['Then', 'you', 'are', 'cast', 'into', 'the', 'Gorge', 'of', 'Eternal', 'Peril', '.']
#parsing: ['Oh', ',', 'I', 'wo

Here is an example of parsing a sentence from **'devset.txt'**.

In [24]:
parse_gram = Pcfg(["S1_tunedE.gr" , "S2_test.gr", "Tagged_Vocab.gr"])
parser = CkyParse(parse_gram, beamsize=0.0001)
sent = 'Who goes there ?'
ce, trees = parser.parse_sentences([sent])
print_tree(trees[0])

(TOP (S2 (_WP (WP Who) (_VBZ (VBZ goes) (_EX (EX there) (_END (END ?) )))) ) )
         TOP               
          |                 
          S2               
          |                 
         _WP               
  ________|____             
 |            _VBZ         
 |    _________|____        
 |   |             _EX     
 |   |          ____|___    
 |   |         |       _END
 |   |         |        |   
 WP VBZ        EX      END 
 |   |         |        |   
Who goes     there      ?  



#reading grammar file: S1_tunedE.gr
#reading grammar file: S2_test.gr
#reading grammar file: Tagged_Vocab.gr
#parsing: ['Who', 'goes', 'there', '?']
#-cross entropy (bits/word): -7.4614


###  [Back to contents](#part-0)

# Part 6. Generating sentences  <a id='part-6'></a>

In [30]:
gen_gram = Pcfg(["S1_tunedE.gr", "S2_test.gr", "Tagged_Vocab.gr"])

gen = PcfgGenerator(gen_gram)
for _ in range(20):
    print(" ".join(gen.generate()))

yes dead depressing to your bunny -- Nee .
where shalt stand me nee 's he have to crossed One .
he back pansy he n't well , yes stay a Banorg strewn three valleys cannot are the Heh shrubberies , proclaim through , of that 's being so !
all not !
Lord taunting to We
Nee could feel left of the Arthur and A shrubbery rode what haw Look All gentle Hurry ? ! .
when 's your sword make feel discovers Father in the legendary spirit ! !
' Ooh .
How excepting ... am for you kill suggests look your nothing Mine labon ' on ... I a sort shut there What get in meeting bloody keeper We -- and single zis ! ' land , Oh , Torment !
No cross confuse Unfortunately guard to Galahad !
Sir , for merger !
dare death of Grenade . )
what me 'm , so lie .
Oh .
Say We .
through !
Who 's you this clever cadeau that such , perpetuating Nine , have I the Patsy shrubbery !
you must Then , words - from that 's What finds .
it feel feast necessary ?
able , and Christ ?


#reading grammar file: S1_tunedE.gr
#reading grammar file: S2_test.gr
#reading grammar file: Tagged_Vocab.gr


###  [Back to contents](#part-0)